In [18]:
import asyncio
import sys
from functools import wraps
from typing import Literal
import litellm
import pandas as pd
from litellm import acompletion
from sklearn.metrics import classification_report, accuracy_score
from tqdm.asyncio import tqdm
from pydantic import BaseModel
from litellm import completion_cost

sys.path.append("../..")

from climateguard.evaluation import (
    plot_cards_confusion_matrix,
    plot_binary_confusion_matrix,
)

## Load the labeled data


In [19]:
# Load the data from the human labeled Excel file
df = pd.read_csv(
    "../../data/raw/4_channels_review_09_2023_09_2024.xlsx - Claims review.csv",
    index_col=0,
)

# Filter the records that have been labeled
df = df[
    # Only keep the records with a ground truth label
    ~df["cards_ground_truth"].isna()
    # Only keep the records with a correctly identified quote since the current labelling UX doesn't allow to identify the correct quotes
    & (df["quote_is_correct"] == "TRUE")
    # Some "commentaire_quote" mention that the identified quote is either incorrect or partially correct, we ignore these records
    & (df["commentaire_quote"].isna())
]

# Only keep the relevant columns
df["is_contrarian_true"] = df["cards_ground_truth"] != "0_accepted"
df["is_contrarian_pred"] = df["cards"] != "0_accepted"
df = df[
    [
        "text",
        "quote",
        "is_contrarian_true",
        "is_contrarian_pred",
        "cards_ground_truth",
        "cards",
    ]
].rename(
    columns={
        "cards_ground_truth": "cards_true",
        "cards": "cards_pred",
        "quote": "quote_true",
    }
)

df

,text,quote_true,is_contrarian_true,is_contrarian_pred,cards_true,cards_pred
id,,,,,,
34a41bf34b35ee91fc147601fb8c21a366a2f568b9060bbb629698dfd9319801,"Jusqu'au trente septembre, détaille Sofia au q...",Il y a beaucoup de soldats poneys menteurs à t...,False,True,0_accepted,5_science_uncertain
0fb8db32982baea27fa4a92220e76331e703d61852c0e8d95550ef6853ffd842,On tente d'échapper à une tempête de pluie d'a...,tempête de pluie d'acide mortelle selon l'inrs...,False,True,0_accepted,1_its_not_happening
4792f93c6614b1e7ef39e301cc6c1d0f4d3d18b9421cc3a9b18aa5c7581c9e02,"Les Français n'arrivent pas à se loger, c'est ...",Ça va de l'interdiction des avions d'affaires ...,True,True,6_proponents_biased,3_impacts_not_bad
74c05fdbca4aeffb643abf0de486b57f8299051d4ef71b58b532791a33aba423,Le réchauffement climatique a été collecté à G...,on ne peut pas faire le lien directement avec ...,True,True,2_humans_not_the_cause,2_humans_not_the_cause
af3a15d298ff642982bcd6c79c21b56623649a140ce3558316dea54ea7ce0f3e,"Vous avez ce livre qui est une bande dessinée,...",la crise écologique va rendre la terre inhospi...,False,True,0_accepted,1_its_not_happening
...,...,...,...,...,...,...
e7d18bd700a5b9d98f38089975d13d7dc9fa873b15ded72e86031b0690309ec7,"Il l'affirme, la piste criminelle est toujours...",les utilisés PFAS dans utilisés la dans fabric...,False,True,0_accepted,5_science_uncertain
58feacdc4a4d31023a87bce158ee902d37076d9f5562f3fe41f768ba815b7b6a,"Aujourd'hui, ce fameux pacte d'immigration qui...",ce tsunami de normes de contraintes de taxe qu...,True,True,4_solutions_harmful_unnecessary,4_solutions_harmful_unnecessary
6d39a0d45c359fd2183c0f1f001b76c4b60d283240e5c34308b71c22c78217ec,"Avec l'aide de nos fournisseurs, si il s'avère...",l'entreprise n'a pas voulu nous dire combien d...,False,True,0_accepted,4_solutions_harmful_unnecessary


In [20]:
plot_cards_confusion_matrix(
    df["cards_true"],
    df["cards_pred"],
    [
        "0_accepted",
        "1_its_not_happening",
        "2_humans_not_the_cause",
        "3_impacts_not_bad",
        "4_solutions_harmful_unnecessary",
        "5_science_uncertain",
        "6_proponents_biased",
        "7_fossil_fuels_needed",
    ],
)

In [21]:
print(
    classification_report(
        df["is_contrarian_true"], df["is_contrarian_pred"], zero_division=0
    )
)
print(classification_report(df["cards_true"], df["cards_pred"], zero_division=0))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00        76
        True       0.62      1.00      0.77       124

    accuracy                           0.62       200
   macro avg       0.31      0.50      0.38       200
weighted avg       0.38      0.62      0.47       200

                                 precision    recall  f1-score   support

                     0_accepted       0.00      0.00      0.00        76
            1_its_not_happening       0.21      0.81      0.33        16
         2_humans_not_the_cause       0.55      0.85      0.67        13
              3_impacts_not_bad       0.33      0.73      0.46        11
4_solutions_harmful_unnecessary       0.57      0.57      0.57        49
            5_science_uncertain       0.50      0.90      0.64        10
            6_proponents_biased       0.68      0.62      0.65        21
          7_fossil_fuels_needed       0.57      1.00      0.73         4

              

## Improve the binary classification


In [22]:
df = df.drop(columns=["is_contrarian_pred", "cards_pred"])

In [23]:
# df = df.iloc[:5].copy()
# df

In [24]:
from openai import AsyncOpenAI

client = AsyncOpenAI()

# Limit concurrent requests to avoid API rate limiting
# (it depends on the model you use and your API tier)
semaphore = asyncio.Semaphore(100)


class MediatreePrediction(BaseModel):
    label_pred: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    cost: float
    claim_pred: str | None = None


class Claim(BaseModel):
    claim: str
    context: str
    analysis: str
    disinformation_score: Literal["very low", "low", "medium", "high"]
    disinformation_category: str
    pro_anti: Literal["pro-écologie", "anti-écologie"]
    speaker: Literal["consensus", "facts", "narrative", "other"]
    contradiction: str
    quote: str


class Claims(BaseModel):
    claims: list[Claim]


async def detect_claims(transcription):
    prompt = f"""Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. 
A partir de cet extrait liste moi tous les faits/opinions environnementaux (claim) uniques qu'il faudrait factchecker. Et pour chaque claim, donne une première analyse si c'est de la désinformation ou non, un score si c'est de la désinformation, ainsi qu'une catégorisation de cette allégation.
Ne sélectionne que les claims sur les thématiques environnementales (changement climatique, transition écologique, énergie, biodiversité, pollution, pesticides, ressources (eau, minéraux, ..) et pas sur les thématiques sociales et/ou économiques
Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois les champs suivants : 

- "claim" - l'allégation à potentiellement vérifier
- "context" - reformulation du contexte dans laquelle cette allégation a été prononcée (maximum 1 paragraphe)
- "analysis" - première analyse du point de vue de l'expert sur le potentiel de désinformation de cette allégation en fonction du contexte
- "disinformation_score" - le score de désinformation (voir plus bas)
- "disinformation_category" - la catégorie de désinformation (voir plus bas)
- "pro_anti" - si l'allégation est plutôt anti-écologie ou pro-écologie
- "speaker" - nom et fonction de la personne qui a prononcé l'allégation si on a l'information (sinon "N/A")
- "contradiction" - si l'allégation a été contestée dans un dialogue, résume la contradiction (sinon "N/A")
- "quote" - la citation exacte qui correspond à l'allégation

Pour les scores "disinformation_score"
- "very low" = pas de problème, l'allégation n'est pas trompeuse ou à risque. pas besoin d'investiguer plus loin
- "low" = allégation qui nécessiterait une vérification et une interrogation, mais sur un sujet peu important et significatif dans le contexte des enjeux écologiques (exemple : les tondeuses à gazon, 
- "medium" = allégation problématique sur un sujet écologique important (scientifique, impacts, élections, politique, transport, agriculture, énergie, alimentation, démocratie ...) , qui nécessiterait vraiment d'être vérifiée, déconstruite, débunkée et interrogée. En particulier pour les opinions fallacieuses
- "high" = allégation grave, en particulier si elle nie le consensus scientifique

Pour les catégories de désinformation "disinformation_category": 
- "consensus" = négation du consensus scientifique
- "facts" = fait à vérifier, à préciser ou contextualiser
- "narrative" = narratif fallacieux ou opinion qui sème le doute (par exemple : "les écolos veulent nous enlever nos libertés")
- "other"

<transcription>
{transcription}
</transcription>"""
    async with semaphore:
        # Retry if errors
        while True:
            try:
                completion = await client.beta.chat.completions.parse(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}],
                    response_format=Claims,
                )
                break
            except (
                Exception
                # litellm.exceptions.RateLimitError,
                # litellm.exceptions.Timeout,
                # litellm.exceptions.APIConnectionError,
                # litellm.exceptions.InternalServerError,
                # litellm.exceptions.ServiceUnavailableError,
            ) as e:
                print(
                    'Caught an exception in "detect_claims", retrying in 10 seconds...'
                )
                print(e)
                await asyncio.sleep(10)

    claims_data = completion.choices[0].message.parsed
    result = [
        claim.model_dump()
        for claim in claims_data.claims
        if claim.pro_anti == "anti-écologie"
    ]

    return result


# Make predictions
df["claims_pred"] = await tqdm.gather(*[detect_claims(text) for text in df["text"]])


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.I

  0%|          | 0/200 [00:00<?, ?it/s]

Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...
Request timed out.
Caught an exception in "detect_claims", retrying in 10 seconds...

  0%|          | 0/200 [00:27<?, ?it/s]


CancelledError: 

In [ ]:
# Is contrarian if we detected a claim
df["is_contrarian_pred"] = df["claims_pred"].str.len() > 0
print(classification_report(df["is_contrarian_true"], df["is_contrarian_pred"]))
plot_binary_confusion_matrix(df["is_contrarian_true"], df["is_contrarian_pred"])

              precision    recall  f1-score   support

       False       0.75      0.16      0.26        76
        True       0.65      0.97      0.78       124

    accuracy                           0.66       200
   macro avg       0.70      0.56      0.52       200
weighted avg       0.69      0.66      0.58       200



In [ ]:
df["claims_pred_high"] = df["claims_pred"].apply(
    lambda claims: [
        claim for claim in claims if claim["disinformation_score"] == "high"
    ]
)
df["is_contrarian_pred"] = df["claims_pred_high"].str.len() > 0
print(classification_report(df["is_contrarian_true"], df["is_contrarian_pred"]))

              precision    recall  f1-score   support

       False       0.58      0.59      0.59        76
        True       0.75      0.74      0.74       124

    accuracy                           0.69       200
   macro avg       0.67      0.67      0.67       200
weighted avg       0.69      0.69      0.69       200



In [52]:
# Actually I realized the descriptions of categories don't help with performance
# It's a bit surprising... maybe the few-shot claim examples are very similar to the test claims?
async def detect_claims(text: str) -> MediatreePrediction:
    system_prompt = """Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. Certains extraits contiennent un passage climatosceptique qui mérite d'être factchecker. Ta tâche est de trouver ce passage climatosceptique (si il existe). Après avoir analyser l'extrait, isole le passage à factchecker.

# Étapes

1. Lire attentivement l'extrait fourni : Se concentrer sur les propos liés au climat, au réchauffement climatique ou aux phénomènes météorologiques extrêmes. Ignorer les discussions non pertinentes (économie, société, etc.).
2. Détecter les éléments potentiellement climatosceptiques. Voilà quelques catégories de désinformation :
   - Négation du changement climatique : Affirme que le réchauffement climatique n'existe pas et que les phénomènes associés, comme la fonte des glaces ou les événements climatiques extrêmes, ne se produisent pas.
   - Rejet de la responsabilité humaine : Soutient que les gaz à effet de serre émis par les activités humaines ne sont pas la cause du changement climatique.
   - Minimisation des impacts : Suggère que les effets du changement climatique ne seront pas graves et pourraient même avoir des avantages.
   - Opposition aux solutions climatiques : Considère que les mesures pour lutter contre le changement climatique sont nuisibles ou inutiles.
   - Critique de la science climatique : Remet en question la fiabilité, la validité et l’objectivité des sciences liées au climat.
   - Attaques contre les défenseurs de l’action climatique : Accuse les scientifiques et les militants d’être alarmistes, biaisés, corrompus ou motivés politiquement.
   - Promotion des énergies fossiles : Défend les combustibles fossiles comme essentiels pour la croissance économique, la prospérité et le maintien du niveau de vie.
4. Extraire le passage climatosceptique. Si aucun passage climatosceptique n’est trouvé : Répondre avec une balise indiquant AUCUN.

# Format de réponse

Fournis le passage climatosceptique au dans des tags XML:

```xml
<passage climatosceptique>
[...]
</passage climatosceptique>
```

Débute ta réponse par "<passage climatosceptique>".

# Exemples

## Exemple sans désinformation

<extrait>
Les climats qui se multiplient, elles sont en poste, ont appelé à cette pause. Non, non seulement il ne faut pas une pause, le réchauffement climatique est en cours. Voilà, enfin, elle aurait bien besoin d'une pause. C'est Valéry, ça devient inquiétant. Écoutez, parler de la fin des voitures thermiques, c'est un impératif économique, puisqu'un plein d'une voiture électrique est beaucoup moins cher qu'un plein d'une voiture à essence. Au début, il faut le dire, le président Pascal va finir par Calais, je crois. Enfin bon, bref, quand il y a un débat sur l'écologie, normalement, les écolos sont plus forts, oui, forcément, sauf quand c'est Marie Toussaint. Or, moi, je connais la proposition du Shift, qui est celle de limiter les vols à Paris. Donc, par et personne, bien sûr, je suis obligé de faire une gratification instantanée. Ce n'est pas une proposition du Shift, je croyais banni. Donc, même dans leurs domaines de prédilection, ils n'arrivent pas. Bon, là, ça va être compliqué. Tout savoir, ça pousse. Cette chanson, c'est ce qui est maintenant, c'est les rouges et plus. Merci beaucoup, Dimitri Bernet, le zapping politique sur Europe 1, à retrouver sur europe1.fr. Sur les réseaux sociaux également, vous nous avez donné tout à l'heure le premier invité de Culture Médias, à partir de neuf heures et demie sur Europe 1, que David Ginola. C'est pour la série au micro. Alors, c'est très intéressant, c'est très marrant, cette série, trouver le nouveau commentateur. En fait, seize voix, mais du commentaire de Joe en août à travers plusieurs villes. Il y a eu déjà à Paris, il va y avoir plein d'autres étapes, Lille, Marseille, et David fait partie du comité de sélection. Et à partir de dix heures, c'est là que le scandale arrive. Inès Reg, vous savez, danse avec les stars, touche à tout, ça pour son nouveau spectacle, c'est avec Jean-Pierre Foucault.
</extrait>
<passage climatosceptique>
AUCUN
</passage climatosceptique>

## Exemple avec désinformation

<extrait>
À ce trafic, hein, mais comme derrière, on n'a ni places de prison ni volonté des magistrats d'embastiller sur des durées longues et dissuasives, on a perdu sur tous les tableaux. Donc, on ne fait qu'augmenter la violence. En réalité, c'est cette politique consistant à taper sur les points de deal. Je ne suis pas contre, sauf que s'il n'y a pas de répression pénale derrière, eh bien, vous ne ferez qu'accroître la violence. Alors que la France est épinglée par l'Europe pour ses dispenses d'interdiction des néonicotinoïdes en 2021 et 2022, l'Office français de la biodiversité demande à ses inspecteurs de ne pas contrôler les arboriculteurs, pourtant grands consommateurs de pesticides, des produits essentiels à leur survie. Illustration en Seine-et-Marne de Florent Ferro et de Sarah. Les néonicotinoïdes ont été bannis de l'agriculture française en 2018. À cause de l'interdiction, cet agriculteur a dû couper tous ses cerisiers. Le champ de cerisiers, notre chambre cerisier, cerisier, mille sept cents arbres ont été arrachés. Finalement, la possibilité de maintenir une production dans la légalité représente un véritable danger pour l'environnement. Ce pesticide représente, malgré tout, un besoin vital pour la production fruitière. Sans cet insecticide, les arbres fruitiers sont infectés par une maladie transmise par un puceron. Certains produits, eh bien, on sera obligé d'abandonner l'arboriculture. Ce serait dommage pour cet agriculteur présent depuis toujours dans la région. La cohabitation entre vergers, produits phytosanitaires et abeilles est plus que possible et il n'est pas incompatible. Le Conseil d'État a jugé cette semaine que les dérogations accordées en 2021 et 2022 concernant l'utilisation de ces néonicotinoïdes...
</extrait>
<passage climatosceptique>
La cohabitation entre vergers, produits phytosanitaires et abeilles est plus que possible et il n'est pas incompatible
</passage climatosceptique>"""
    response = await acompletion(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"<extrait>\n{text.strip()}\n</extrait>"},
            # {
            #     "role": "assistant",
            #     "content": "<passage climatosceptique>",
            #     "prefix": True,
            # },
        ],
        model="gpt-4o-mini",
        max_tokens=1000,
        temperature=0,
        stop=["</p"],  # Same as "</passage climatosceptique>" with fewer tokens
    )

    # Parse the response
    response_content: str = response.choices[0].message.content
    claim = response_content.split("<passage climatosceptique>")[1].strip()

    return claim if claim != "AUCUN" else None
    return MediatreePrediction(
        prompt_tokens=response.usage.prompt_tokens,
        completion_tokens=response.usage.completion_tokens,
        total_tokens=response.usage.total_tokens,
        cost=completion_cost(response),
        claim_pred=claim if claim != "AUCUN" else None,
        label_pred="",
    )


# Make predictions
df["claims_pred"] = await tqdm.gather(*[detect_claims(text) for text in df["text"]])

100%|██████████| 200/200 [00:07<00:00, 26.26it/s] 


In [53]:
df["is_contrarian_pred"] = ~df["claims_pred"].isna()
print(classification_report(df["is_contrarian_true"], df["is_contrarian_pred"]))

              precision    recall  f1-score   support

       False       0.83      0.25      0.38        76
        True       0.68      0.97      0.80       124

    accuracy                           0.69       200
   macro avg       0.75      0.61      0.59       200
weighted avg       0.73      0.69      0.64       200



In [33]:
df

,text,quote_true,is_contrarian_true,cards_true,claims_pred,is_contrarian_pred
id,,,,,,
34a41bf34b35ee91fc147601fb8c21a366a2f568b9060bbb629698dfd9319801,"Jusqu'au trente septembre, détaille Sofia au q...",Il y a beaucoup de soldats poneys menteurs à t...,False,0_accepted,Il y a beaucoup de soldats poneys menteurs à t...,True
0fb8db32982baea27fa4a92220e76331e703d61852c0e8d95550ef6853ffd842,On tente d'échapper à une tempête de pluie d'a...,tempête de pluie d'acide mortelle selon l'inrs...,False,0_accepted,None,False
4792f93c6614b1e7ef39e301cc6c1d0f4d3d18b9421cc3a9b18aa5c7581c9e02,"Les Français n'arrivent pas à se loger, c'est ...",Ça va de l'interdiction des avions d'affaires ...,True,6_proponents_biased,la vérité est qu'une certaine écologie oscille...,True
74c05fdbca4aeffb643abf0de486b57f8299051d4ef71b58b532791a33aba423,Le réchauffement climatique a été collecté à G...,on ne peut pas faire le lien directement avec ...,True,2_humans_not_the_cause,on ne peut pas faire le lien directement avec ...,True
af3a15d298ff642982bcd6c79c21b56623649a140ce3558316dea54ea7ce0f3e,"Vous avez ce livre qui est une bande dessinée,...",la crise écologique va rendre la terre inhospi...,False,0_accepted,None,False
...,...,...,...,...,...,...
e7d18bd700a5b9d98f38089975d13d7dc9fa873b15ded72e86031b0690309ec7,"Il l'affirme, la piste criminelle est toujours...",les utilisés PFAS dans utilisés la dans fabric...,False,0_accepted,« Il n'y a pas de distinction entre les PFAS. ...,True
58feacdc4a4d31023a87bce158ee902d37076d9f5562f3fe41f768ba815b7b6a,"Aujourd'hui, ce fameux pacte d'immigration qui...",ce tsunami de normes de contraintes de taxe qu...,True,4_solutions_harmful_unnecessary,le fanatisme vert et le pacte vert qu'il souti...,True
6d39a0d45c359fd2183c0f1f001b76c4b60d283240e5c34308b71c22c78217ec,"Avec l'aide de nos fournisseurs, si il s'avère...",l'entreprise n'a pas voulu nous dire combien d...,False,0_accepted,None,False


## Improve the CARDS classification
